In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
%matplotlib inline

# Program that gathers data from whatsapp .txt

architecture :
    - files in work_directory/original_data/
    - files produced saved in work_directory/data/
    
###### Potential improvments : automatic creation of directories 

## To change : nom_conv

In [18]:
nom_conv = 'Conv_1' # the name of the whatsapp conversation without the file extension name (ex: in "Whatsapp.txt" indicate "Whatsapp")

In [19]:
conv = './original data/' + nom_conv + '.txt' # access to file

In [20]:
def Datepattern(s):
    pattern = '^([0-2][0-9]|(3)[0-1])(\/)(((0)[0-9])|((1)[0-2]))(\/)(\d{2}|\d{4}) à ([0-9][0-9]):([0-9][0-9]) -'
    result = re.match(pattern, s)
    if result: # If it recognizes a "(date) à (hour) -", returns the value "True" 
        return True
    return False # If it doesn't, return the value "False"

# Works with french messages where the separator in the txt file is "à". Change that in the code if it is not the case for you
# Works for messages such as : 01/01/2020 à 00:00 - Author_1: message
# If you have that type : 01/01/2020, 00:00 - Author_1: message, change "à" by ","

In [21]:
def Auteurpattern(s):
    patterns = [
        '([\w]+):',                        # Recognizes one word followed by ":"
        '([\w]+[\s]+[\w]+):',              # Recognizes two words separated by a space and followed by ":"
        '([\w]+[\s]+[\w]+[\s]+[\w]+):',    # Recognizes two words separated by a space and followed by ":"
    ]
    pattern = '^' + '|'.join(patterns)
    result = re.match(pattern, s)
    if result:
        return True # same thing as above
    return False

In [22]:
def getData(line):
    splitLine = line.split(' - ') # splits at the dash, giving us two elements, the date and hour and the author and the message
    dateTime = splitLine[0] # we take the date and time and store the
    date, time = dateTime.split('à ') # we split at "à" because of the french version
    
    message = ' '.join(splitLine[1:]) # We store the second element of the list as the message
    
    if Auteurpattern(message): # We look for an author at the beginning
        splitMessage = message.split(': ') # If we find one, we split at the column
        author = splitMessage[0] # the first element is the author
        message = ' '.join(splitMessage[1:]) # the second is the message itself
    else: # if it doesn't start with an author, it's an automatic message
        author = None # the non-author is stored
    return date, time, author, message

In [23]:
Data = [] # to save for later
fp = open(conv, encoding="utf-8")
fp.readline() #Reads the first line without applying any operations on it (it's usually a line that says "data is encrypted")
    
temp = [] # We will store temporarly messages, in case there is a break in the line
date, time, author = None, None, None # initialization of variables

while True: #We could also use an if statement and stop with the number of lines of the txt file
    line = fp.readline() # we read line by line
    if not line: # Stops when we reach the end of the file
        break
    line = line.strip() # .strip() removes the potential blank spaces and hidden caracters
    if Datepattern(line): # If it starts by a date, it's a new message
        if len(temp) > 0: # If the buffer contains elements, that means we have stored a message
            Data.append([date, time, author, ' '.join(temp)]) # so we save data, since we're in a new message
        temp.clear() # clearing the buffer
        date, time, author, message = getData(line) # identifying elements of the message
        temp.append(message) # saving it in case there is a new line
    else: # If it doesn't start with a date, it's following the previous message so we add it in the buffer
        temp.append(line) 

In [24]:
df = pd.DataFrame(Data, columns=['Date', 'Heure', 'Auteur', 'Message']) #Finally creating the dataframe

fichier_clean = './original data/' + nom_conv + '_clean.csv'  #naming a new intermediary file

#Saving the file for next time
df.to_csv(fichier_clean, encoding='utf-8', index=False)
#df.head(10)

#### If you don't want the last part that adds a column, just stop the code here.

In [25]:
df = pd.read_csv(fichier_clean, parse_dates = [["Date","Heure"]] ,dayfirst=True, na_filter=False) # reads the file previously created

In [26]:
auteurs = df["Auteur"].unique() #stores all the authors

In [27]:
frames = list(range(len(auteurs))) # we create a list that will contain data frames
for i in range(len(auteurs)):
    temp = df[df["Auteur"]== auteurs[i]].reset_index(drop=True) #stores a part of the datafram linked to authors
    total_mssg = temp["Message"].count() #counts the number of messages sent by that author
    for j in range(total_mssg): # adds a column and adds a counter of messages
        temp.loc[j,'Compteur'] = j+1
    frames[i]=temp #stores the different parts to unify again the dataframe after


In [28]:
fichier_compte = pd.concat(frames) # recreates the data frame
fichier_compte = fichier_compte.sort_values(by=['Date_Heure','Compteur']) # sort it first by time, and then by the "count" column we created

In [29]:
fichier_compte.to_csv( './data/' + nom_conv + '_compteur.csv', encoding='utf-8', index=False) #saves the dataframe in a csv file

### Credits : The first part of the code by Samir Sheriff can be found here : https://towardsdatascience.com/build-your-own-whatsapp-chat-analyzer-9590acca9014. I changed a few things (in order to adapt it to French chats) and added a few others.